# Data Science for Good - Text classification using PySpark ML

This Notebook utilizes Machine Learning with <code>PySpark</code> to categorize disaster tweets. This is created for the [Kaggle competition](https://www.kaggle.com/c/nlp-getting-started).

In [0]:
#!pip install pyspark

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))


### Importing necessary libraries

In [0]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
from pyspark.sql.functions import col, udf,regexp_replace,isnull
from pyspark.sql.types import StringType,IntegerType
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression, DecisionTreeClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

### Create a Spark session

In [0]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

## Exploring Data Analysis
### Load the data files

In [0]:
#filepath = '/kaggle/input/nlp-getting-started'
#sdf_train = spark.read.csv(f'{filepath}/train.csv', header = True, inferSchema = True)
#sdf_test = spark.read.csv(f'{filepath}/test.csv', inferSchema=True, header=True)

#sdf_sample_submission = spark.read.csv(f'{filepath}/sample_submission.csv', 
#                                       inferSchema=True, header=True)
#sdf_train.printSchema()

In [0]:
sdf_train = spark.read.csv('/FileStore/tables/news_ft/news_train.csv',inferSchema=True,header=True)
sdf_test = spark.read.csv('/FileStore/tables/news_ft/news_test.csv',inferSchema=True,header=True)

#sdf_sample_submission = spark.read.csv(f'{filepath}/sample_submission.csv', 
  #                                     inferSchema=True, header=True)
sdf_train.printSchema()

root
-- _c0: string (nullable = true)
-- Unnamed: 0: string (nullable = true)
-- title: string (nullable = true)
-- text: string (nullable = true)
-- subject: string (nullable = true)
-- date: string (nullable = true)
-- target: integer (nullable = true)
-- id: integer (nullable = true)

**Predictor variables:** id, keyword, location, text

**Outcome variable:** target

Let's take a look at how the data looks.  
Pandas data frame is better than Spark DataFrame show() function.

In [0]:
import pandas as pd
pd.DataFrame(sdf_train.take(5), columns=sdf_train.columns)

,_c0,Unnamed: 0,title,text,subject,date,target,id
0,1,1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17,0,2
1,3,3,Trump Is So Obsessed He Even Has Obama’s Name...,On Christmas day Donald Trump announced that h...,News,29-Dec-17,0,4
2,6,6,Fresh Off The Golf Course Trump Lashes Out At...,Donald Trump spent a good portion of his day a...,News,23-Dec-17,0,7
3,7,7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,23-Dec-17,0,8
4,8,8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,22-Dec-17,0,9


In [0]:
print("Training Data Record Count:",sdf_train.count())
print("Test Data Record Count:",sdf_test.count())

Training Data Record Count: 35908
Test Data Record Count: 8978

In [0]:
sdf_train.toPandas().groupby(['target']).size()

Out[64]: target
0.0 18746
1.0 17153
dtype: int64

The data is well balanced.

## Data Pre-processing

In [0]:
ml_df = sdf_train.select("id","text","target")
ml_df.show(5)

+---+--------------------+------+
 id| text|target|
+---+--------------------+------+
 2|House Intelligenc...| 0|
 4|On Christmas day ...| 0|
 7|Donald Trump spen...| 0|
 8|In the wake of ye...| 0|
 9|Many people have ...| 0|
+---+--------------------+------+
only showing top 5 rows

### Cleaning the dataset
#### Drop null values

In [0]:
ml_df = ml_df.dropna()
ml_df.count()

Out[66]: 35899

#### Removing numbers from the tweets

In [0]:
ml_df = ml_df.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
ml_df.show(5)

<>:1: DeprecationWarning: invalid escape sequence \d
<>:1: DeprecationWarning: invalid escape sequence \d
<command-3170796558011042>:1: DeprecationWarning: invalid escape sequence \d
 ml_df = ml_df.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
+---+--------------------+------+--------------------+
 id| text|target| only_str|
+---+--------------------+------+--------------------+
 2|House Intelligenc...| 0|House Intelligenc...|
 4|On Christmas day ...| 0|On Christmas day ...|
 7|Donald Trump spen...| 0|Donald Trump spen...|
 8|In the wake of ye...| 0|In the wake of ye...|
 9|Many people have ...| 0|Many people have ...|
+---+--------------------+------+--------------------+
only showing top 5 rows

#### Segregating the words from the tweet

In [0]:
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")
raw_words = regex_tokenizer.transform(ml_df)
raw_words.show(5)

+---+--------------------+------+--------------------+--------------------+
 id| text|target| only_str| words|
+---+--------------------+------+--------------------+--------------------+
 2|House Intelligenc...| 0|House Intelligenc...|[house, intellige...|
 4|On Christmas day ...| 0|On Christmas day ...|[on, christmas, d...|
 7|Donald Trump spen...| 0|Donald Trump spen...|[donald, trump, s...|
 8|In the wake of ye...| 0|In the wake of ye...|[in, the, wake, o...|
 9|Many people have ...| 0|Many people have ...|[many, people, ha...|
+---+--------------------+------+--------------------+--------------------+
only showing top 5 rows

#### Removing the stop words from raw words

In [0]:
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
words_df = remover.transform(raw_words)
words_df.select("id","words","target","filtered").show(5, truncate=False)

+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### Create a features column from the words

In [0]:
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(words_df)
countVectorizer_train = model.transform(words_df)
countVectorizer_train = countVectorizer_train.withColumn("label",col('target'))
countVectorizer_train.show(5)

+---+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
 id| text|target| only_str| words| filtered| features|label|
+---+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
 2|House Intelligenc...| 0|House Intelligenc...|[house, intellige...|[house, intellige...|(104745,[0,1,2,7,...| 0|
 4|On Christmas day ...| 0|On Christmas day ...|[on, christmas, d...|[christmas, day, ...|(104745,[0,2,4,8,...| 0|
 7|Donald Trump spen...| 0|Donald Trump spen...|[donald, trump, s...|[donald, trump, s...|(104745,[0,2,10,1...| 0|
 8|In the wake of ye...| 0|In the wake of ye...|[in, the, wake, o...|[wake, yet, anoth...|(104745,[0,2,4,6,...| 0|
 9|Many people have ...| 0|Many people have ...|[many, people, ha...|[many, people, ra...|(104745,[0,2,3,4,...| 0|
+---+--------------------+------+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [0]:
countVectorizer_train.select('text','words','filtered','features','target').show()

+--------------------+--------------------+--------------------+--------------------+------+
 text| words| filtered| features|target|
+--------------------+--------------------+--------------------+--------------------+------+
House Intelligenc...|[house, intellige...|[house, intellige...|(104745,[0,1,2,7,...| 0|
On Christmas day ...|[on, christmas, d...|[christmas, day, ...|(104745,[0,2,4,8,...| 0|
Donald Trump spen...|[donald, trump, s...|[donald, trump, s...|(104745,[0,2,10,1...| 0|
In the wake of ye...|[in, the, wake, o...|[wake, yet, anoth...|(104745,[0,2,4,6,...| 0|
Many people have ...|[many, people, ha...|[many, people, ra...|(104745,[0,2,3,4,...| 0|
A centerpiece of ...|[a, centerpiece, ...|[centerpiece, don...|(104745,[0,4,5,11...| 0|
Republicans are w...|[republicans, are...|[republicans, wor...|(104745,[0,4,7,15...| 0|
Republicans have ...|[republicans, hav...|[republicans, sev...|(104745,[0,1,2,5,...| 0|
The media has bee...|[the, media, has,...|[media, talking, ...|(104745,[0,1,2,16...| 0|
Abigail Disney is...|[abigail, disney,...|[abigail, disney,...|(104745,[0,4,5,7,...| 0|
Donald Trump just...|[donald, trump, j...|[donald, trump, s...|(104745,[0,1,5,8,...| 0|
Right now the who...|[right, now, the,...|[right, whole, wo...|(104745,[0,1,2,4,...| 0|
In this #METOO mo...|[in, this, metoo,...|[metoo, moment, m...|(104745,[2,4,6,12...| 0|
Alabama is a noto...|[alabama, is, a, ...|[alabama, notorio...|(104745,[1,4,6,8,...| 0|
A backlash ensued...|[a, backlash, ens...|[backlash, ensued...|(104745,[0,1,2,11...| 0|
Ronald Reagan is ...|[ronald, reagan, ...|[ronald, reagan, ...|(104745,[0,2,4,5,...| 0|
Donald Trump held...|[donald, trump, h...|[donald, trump, h...|(104745,[0,4,11,1...| 0|
When Sen. Al Fran...|[when, sen, al, f...|[sen, al, franken...|(104745,[0,1,4,5,...| 0|
New questions are...|[new, questions, ...|[new, questions, ...|(104745,[0,2,3,4,...| 0|
On Wednesday Dona...|[on, wednesday, d...|[wednesday, donal...|(104745,[0,2,4,6,...| 0|
+--------------------+--------------------+--------------------+--------------------+------+
only showing top 20 rows

### Separate the Train and Validation Data

In [0]:
(train, validate) = countVectorizer_train.randomSplit([0.8, 0.2],seed = 97435)

### Test Data

In [0]:
trainData = countVectorizer_train

#cleaning and preparing the test data
testData = sdf_test.select("id","text").dropna()
testData = testData.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
regex_tokenizer = RegexTokenizer(inputCol="only_str", outputCol="words", pattern="\\W")  #Extracting raw words
testData = regex_tokenizer.transform(testData)
remover = StopWordsRemover(inputCol="words", outputCol="filtered") #Removing stop words
testData = remover.transform(testData)
cv = CountVectorizer(inputCol="filtered", outputCol="features")
model = cv.fit(testData)
countVectorizer_test = model.transform(testData)
testData = countVectorizer_test
testData.show(5)

<>:5: DeprecationWarning: invalid escape sequence \d
<>:5: DeprecationWarning: invalid escape sequence \d
<command-3170796558011053>:5: DeprecationWarning: invalid escape sequence \d
 testData = testData.withColumn("only_str",regexp_replace(col('text'), '\d+', ''))
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
 id| text| only_str| words| filtered| features|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
 1|Donald Trump just...|Donald Trump just...|[donald, trump, j...|[donald, trump, c...|(57648,[0,2,4,5,7...|
 3|On Friday it was ...|On Friday it was ...|[on, friday, it, ...|[friday, revealed...|(57648,[0,4,5,11,...|
 5|Pope Francis used...|Pope Francis used...|[pope, francis, u...|[pope, francis, u...|(57648,[0,1,4,5,1...|
 6|The number of cas...|The number of cas...|[the, number, of,...|[number, cases, c...|(57648,[4,21,27,3...|
 10|Just when you mig...|Just when you mig...|[just, when, you,...|[might, thought, ...|(57648,[0,2,4,10,...|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows

# Machine Learning Prediction Models
## Naive Bayes Classifier

In [0]:
nb = NaiveBayes(modelType="multinomial",labelCol="label", featuresCol="features")
nbModel = nb.fit(train)
nb_predictions = nbModel.transform(validate)

In [0]:
nbEval = BinaryClassificationEvaluator()
print('Test Area Under ROC', nbEval.evaluate(nb_predictions))

Test Area Under ROC 0.5177479222708233

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
nb_accuracy = evaluator.evaluate(nb_predictions)
print("Accuracy of NaiveBayes is = %g"% (nb_accuracy))

## Logistic Regression Model

In [0]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'target', maxIter=10)
lrModel = lr.fit(train)

We can obtain the coefficients by using <code>LogisticRegressionModel</code>’s attributes.

In [0]:
import matplotlib.pyplot as plt
import numpy as np

beta = np.sort(lrModel.coefficients)
plt.plot(beta)
plt.ylabel('Beta Coefficients')
plt.show()

Summarize the model

In [0]:
trainingSummary = lrModel.summary
lrROC = trainingSummary.roc.toPandas()

plt.plot(lrROC['FPR'],lrROC['TPR'])
plt.ylabel('False Positive Rate')
plt.xlabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()

print('Training set areaUnderROC: ' + str(trainingSummary.areaUnderROC))

Training set areaUnderROC: 0.9997921143999654

### Precision and recall

In [0]:
pr = trainingSummary.pr.toPandas()
plt.plot(pr['recall'],pr['precision'])
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()

### Validate the model

In [0]:
lrPreds = lrModel.transform(validate)
lrPreds.select('id','prediction').show(5)

+---+----------+
 id|prediction|
+---+----------+
 11| 0.0|
 25| 0.0|
 29| 0.0|
 32| 0.0|
 47| 0.0|
+---+----------+
only showing top 5 rows

### Evaluate the Logistic Regression model

In [0]:
lrEval = BinaryClassificationEvaluator()
print('Test Area Under ROC', lrEval.evaluate(lrPreds))

Test Area Under ROC 0.9954358483682711

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
lr_accuracy = evaluator.evaluate(lrPreds)
print("Accuracy of Logistic Regression is = %g"% (lr_accuracy))

Accuracy of Logistic Regression is = 0.98085

## Decision Tree Classifier

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'target', maxDepth = 3)
dtModel = dt.fit(train)
dtPreds = dtModel.transform(validate)
dtPreds.show(5)
#dtPreds.select('age', 'job', 'label', 'rawPrediction', 'prediction', 'probability').show(10)

### Evaluate the Decision Tree model

One simple decision tree performed poorly because it is too weak given the range of different features. The prediction accuracy of decision trees can be improved by Ensemble methods, such as Random Forest and Gradient-Boosted Tree.

In [0]:
dtEval = BinaryClassificationEvaluator()
dtROC = dtEval.evaluate(dtPreds, {dtEval.metricName: "areaUnderROC"})
print("Test Area Under ROC: " + str(dtROC))

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
dt_accuracy = evaluator.evaluate(dtPreds)
print("Accuracy of Decision Trees is = %g"% (dt_accuracy))

### Make Predictions based on Decision Tree Model

In [0]:
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'target', maxDepth = 3)
dtModel = dt.fit(trainData)
dtPreds = dtModel.transform(testData)
dtPreds.show(5)

In [0]:
dtPreds.select('id','prediction').withColumnRenamed('prediction','target').write.csv('dt_output.csv',header=True)

## Random Forest Classifier

In [0]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'target')
rfModel = rf.fit(train)
rfPreds = rfModel.transform(validate)
rfPreds.select('id', 'rawPrediction', 'prediction', 'probability').show(10)

### Evaluate the Random Forest Classifier

In [0]:
rfEval = BinaryClassificationEvaluator()
rfROC = rfEval.evaluate(rfPreds, {rfEval.metricName: "areaUnderROC"})
print("Test Area Under ROC: " + str(rfROC))

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
rf_accuracy = evaluator.evaluate(rfPreds)
print("Accuracy of Random Forests is = %g"% (rf_accuracy))

### Make Predictions using the Model

In [0]:
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'target')
rfModel = rf.fit(trainData)
rfPreds = rfModel.transform(testData)
#rfPreds.select('id', 'rawPrediction', 'prediction', 'probability').show(10)

In [0]:
#rfPreds.select('id','prediction').withColumnRenamed('prediction','target').write.csv('rf_output',header=True,mode='overwrite')

## Gradient Boosting Classifier

In [0]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
gbtPreds = gbtModel.transform(validate)
gbtPreds.show(5)

### Evaluate the Gradient-Boosted Tree Classifier

In [0]:
gbtEval = BinaryClassificationEvaluator()
gbtROC = gbtEval.evaluate(gbtPreds, {gbtEval.metricName: "areaUnderROC"})
print("Test Area Under ROC: " + str(gbtROC))

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
gb_accuracy = evaluator.evaluate(gbtPreds)
print("Accuracy of GBT is = %g"% (gb_accuracy))

In [0]:
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(trainData)
gbtPreds = gbtModel.transform(testData)
gbtPreds.select('id','prediction').show(5)

In [0]:
#gbtPreds.select('id','prediction').withColumnRenamed('prediction','target').write.csv('gbt_output.csv',header=True,,mode='overwrite')